<a href="https://colab.research.google.com/github/Casssiee/Q3-Budget-Data-Exercise/blob/main/budget_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Budget Data Cleaning – Imputation Policy

Imputation logic implemented:

## 1) Detail rows (no `Total` in any dimension)

**0-rule (structural zeros):**
- If **Actual** and **Forecast** are both missing (`#VALUE!`) **and** **Budget = 0** for that row, treat the row as **no-load** and set **Actual = 0** and **Forecast = 0**.

**Totals-constrained fill (preferred):**
- If a (School, Course) rollup exists where **Funding Category = `Total`**, fill missing detail by allocating the remainder:
  - `remainder = Total - sum(known funding categories)`
- If multiple cells are missing in the same rollup, use **regression predictions as weights** to split the remainder.

**Regression fallback:**
- If no usable (School, Course) total exists for that rollup, fill missing detail using the **regression prediction**.

## 2) Aggregation rows (any dimension contains `Total`, incl. `School = Faculty Total`)
- If an aggregation cell is `#VALUE!`, fill it as the **sum of its child rows** (after detail backfill).
- **Do not overwrite** any existing (non-missing) totals — only fill totals where the cell is missing.


In [1]:
from google.colab import files

print("Please upload 'Analytics Specialist Interview Exercise.xlsx")
uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Please upload 'Analytics Specialist Interview Exercise.xlsx


Saving Analytics Specialist Interview Exercise.xlsx to Analytics Specialist Interview Exercise.xlsx
User uploaded file "Analytics Specialist Interview Exercise.xlsx" with length 89441 bytes


In [2]:
import numpy as np
import pandas as pd

INPUT_XLSX  = "Analytics Specialist Interview Exercise.xlsx"
SHEET_NAME  = "Data for exercise"
OUTPUT_XLSX = "full_corrected_regression_totals.xlsx"

DIM = ["School", "Course", "Funding Category"]


In [3]:
def clean_headers(cols):
    return [str(c).replace("\n", " ").strip() for c in cols]

def load_dataset(path=INPUT_XLSX, sheet_name=SHEET_NAME):
    df = pd.read_excel(path, sheet_name=sheet_name, na_values=["#VALUE!"])
    df.columns = clean_headers(df.columns)
    measures = [c for c in df.columns if c not in DIM]
    for m in measures:
        df[m] = pd.to_numeric(df[m], errors="coerce")
    return df, measures

def add_grain_flags(df):
    out = df.copy()
    out["is_faculty_total"] = out["School"].eq("Faculty Total")
    out["is_course_total"]  = out["Course"].eq("Total")
    out["is_funding_total"] = out["Funding Category"].eq("Total")
    out["is_total_row"]     = out[["is_faculty_total","is_course_total","is_funding_total"]].any(axis=1)
    out["is_detail"]        = ~out["is_total_row"]
    return out


In [4]:
def ridge_fit_beta(X, y, alpha=10.0):
    """Closed-form ridge regression (intercept not penalized)."""
    XtX = X.T @ X
    I = np.eye(X.shape[1])
    I[0, 0] = 0.0
    return np.linalg.solve(XtX + alpha * I, X.T @ y)

def fill_missing_totals_from_detail(df, measures):
    """Fill ONLY missing totals (NaN) using sums of detail rows. Do not overwrite existing totals."""
    detail = df[df["is_detail"]].copy()

    g_sc_f = detail.groupby(["School","Course","Funding Category"])[measures].sum(min_count=1)
    g_sc   = detail.groupby(["School","Course"])[measures].sum(min_count=1)
    g_sf   = detail.groupby(["School","Funding Category"])[measures].sum(min_count=1)
    g_s    = detail.groupby(["School"])[measures].sum(min_count=1)
    g_cf   = detail.groupby(["Course","Funding Category"])[measures].sum(min_count=1)
    g_c    = detail.groupby(["Course"])[measures].sum(min_count=1)
    g_f    = detail.groupby(["Funding Category"])[measures].sum(min_count=1)
    g_all  = detail[measures].sum(min_count=1)

    def expected(row):
        s, c, f = row["School"], row["Course"], row["Funding Category"]
        fac = (s == "Faculty Total")
        ct  = (c == "Total")
        ft  = (f == "Total")

        if not fac:
            if (not ct) and (not ft): return g_sc_f.loc[(s,c,f)]
            if (not ct) and ft:       return g_sc.loc[(s,c)]
            if ct and (not ft):       return g_sf.loc[(s,f)]
            return g_s.loc[s]
        else:
            if (not ct) and (not ft): return g_cf.loc[(c,f)]
            if (not ct) and ft:       return g_c.loc[c]
            if ct and (not ft):       return g_f.loc[f]
            return g_all

    total_idx = df.index[df["is_total_row"]]
    exp_vals = df.loc[total_idx, DIM].apply(expected, axis=1, result_type="expand")
    exp_vals.columns = measures

    for m in measures:
        df.loc[total_idx, m] = df.loc[total_idx, m].fillna(exp_vals[m])
    return df

def impute(df, measures, alpha=10.0, neg_remainder_tolerance=2.0):
    """Impute detail first, then fill missing aggregation cells. Returns (corrected_df, audit)."""
    df = add_grain_flags(df)

    # --- Robust rule: if Actual & Forecast are missing and Budget is 0, treat Actual/Forecast as 0 (detail rows only) ---
    def _pick_col(keyword: str):
        cands = [c for c in measures if keyword.lower() in c.lower()]
        return cands[0] if cands else None

    actual_col = _pick_col("actual")
    forecast_col = _pick_col("forecast")
    budget_col = _pick_col("budget")

    filled_by_zero_rule = 0
    if actual_col and forecast_col and budget_col:
        zero_mask = (
            df["is_detail"]
            & df[actual_col].isna()
            & df[forecast_col].isna()
            & df[budget_col].fillna(0).eq(0)
        )
        filled_by_zero_rule = int(zero_mask.sum())
        if filled_by_zero_rule:
            df.loc[zero_mask, actual_col] = 0.0
            df.loc[zero_mask, forecast_col] = 0.0

    is_detail = df["is_detail"].to_numpy()

    totals_sc = (
        df.loc[(~df["is_faculty_total"]) & (~df["is_course_total"]) & (df["is_funding_total"]),
               ["School", "Course"] + measures]
          .drop_duplicates(["School", "Course"])
          .set_index(["School", "Course"])
    )

    detail_idx = df.index[is_detail]
    X_df = pd.get_dummies(df.loc[detail_idx, DIM], columns=DIM, drop_first=True)
    X = np.column_stack([np.ones(len(X_df)), X_df.to_numpy(dtype=float)])

    preds = {}
    for m in measures:
        y_raw = df.loc[detail_idx, m]
        obs = y_raw.notna().to_numpy()
        if obs.sum() == 0:
            preds[m] = pd.Series(0.0, index=detail_idx)
            continue
        y = np.log1p(y_raw.to_numpy(dtype=float)[obs])
        beta = ridge_fit_beta(X[obs], y, alpha=alpha)
        yhat = np.expm1(X @ beta)
        preds[m] = pd.Series(np.clip(yhat, 0, None), index=detail_idx)

    filled_by_total = {m: 0 for m in measures}
    filled_by_reg   = {m: 0 for m in measures}

    # 1) Detail: fill using Total - sum(rest)
    for (school, course), idxs in df.loc[is_detail].groupby(["School", "Course"]).groups.items():
        if (school, course) not in totals_sc.index:
            continue
        idxs = list(idxs)
        for m in measures:
            total_val = totals_sc.loc[(school, course), m]
            if pd.isna(total_val):
                continue
            vals = df.loc[idxs, m]
            miss = vals.isna()
            if miss.sum() == 0:
                continue

            remainder = total_val - vals.sum(min_count=1)
            if pd.isna(remainder):
                continue
            if remainder < 0 and abs(remainder) <= neg_remainder_tolerance:
                remainder = 0.0
            elif remainder < 0:
                continue

            miss_rows = vals[miss].index
            prior = preds[m].loc[miss_rows]
            prior_sum = prior.sum()
            alloc = (remainder / len(miss_rows)) if prior_sum <= 0 else (remainder * (prior / prior_sum))
            df.loc[miss_rows, m] = np.array(alloc)
            filled_by_total[m] += int(len(miss_rows))

    # 2) Detail: remaining missing -> regression
    for m in measures:
        mask = df["is_detail"] & df[m].isna()
        if mask.any():
            rows = df.index[mask]
            df.loc[rows, m] = preds[m].loc[rows].values
            filled_by_reg[m] += int(len(rows))

    # 3) Aggregations: fill missing totals from sums of detail rows (do not overwrite existing totals)
    df = fill_missing_totals_from_detail(df, measures)

    corrected = df[DIM + measures].copy()
    audit = {"filled_by_zero_rule": filled_by_zero_rule, "filled_by_total": filled_by_total, "filled_by_regression": filled_by_reg}
    return corrected, audit


In [6]:
# ---- RUN ----
df, measures = load_dataset(INPUT_XLSX, SHEET_NAME)
corrected, audit = impute(df, measures)

with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    corrected.to_excel(writer, index=False, sheet_name="full_corrected")

print("Wrote:", OUTPUT_XLSX)
print("\nAudit (cells filled in DETAIL):")
print("  Zero-rule (Actual+Forecast missing & Budget=0):", audit.get("filled_by_zero_rule", 0))
print("  By totals constraint:", audit["filled_by_total"])
print("  By regression fallback:", audit["filled_by_regression"])

detail_mask = (
    (corrected["School"] != "Faculty Total")
    & (corrected["Course"] != "Total")
    & (corrected["Funding Category"] != "Total")
)
print("\nRemaining NaNs in DETAIL rows (should be 0):")
print(corrected.loc[detail_mask, measures].isna().sum().to_string())

print("\nRemaining NaNs in AGGREGATION rows (should be 0):")
print(corrected.loc[~detail_mask, measures].isna().sum().to_string())


Wrote: full_corrected_regression_totals.xlsx

Audit (cells filled in DETAIL):
  Zero-rule (Actual+Forecast missing & Budget=0): 159
  By totals constraint: {'Actuals at end of August': 4, 'Forecast Q3 FY24': 4, 'Budget 2024-28': 75}
  By regression fallback: {'Actuals at end of August': 0, 'Forecast Q3 FY24': 0, 'Budget 2024-28': 63}

Remaining NaNs in DETAIL rows (should be 0):
Actuals at end of August    0
Forecast Q3 FY24            0
Budget 2024-28              0

Remaining NaNs in AGGREGATION rows (should be 0):
Actuals at end of August    0
Forecast Q3 FY24            0
Budget 2024-28              0


In [7]:
# Download the file to local folder
from google.colab import files

files.download('full_corrected_regression_totals.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>